## Data download

The aim is to synthetically create a non-gridded dataset of sea level pressure anomalies. 

To obtain sea level pressure anomalies we first need to get:
1. Climatologies of mean sea level pressure (`msl`/`psl`) over the period 1979--2011 (following IceNet)
2. `msl` data for the training period

Anomalies are then calculated by subtracting the climatological mean for each calendar month from the `msl` data.

In [ ]:
# Set the base path where data should be stored. This should be the same as what you use in your IceStationZebra config.
base_path = "/LOCAL/PATH/WHERE/YOU/WANT/TO/STORE/DATA/"

In [ ]:
import os
from pathlib import Path

data_path = (Path(base_path) / "data" / "notebooks" / "extract_anomalies").resolve()
data_path.mkdir(parents=True, exist_ok=True)
current_directory = Path.cwd()

### Download monthly means (1979-2011)

We'll download from the CDS dataset [ERA5 monthly averaged data on single levels from 1940 to present](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-monthly-means?tab=download). We'll use the CDS API directly, as it is not currently clear how to do so with Anemoi. To use the CDS API you need to have the `cdsapi` package installed and have a `.cdsapirc` file set up under your `$HOME`.

Download only data for the Southern hemisphere. You can do the same for the Northern hemisphere by changing the `area` key to `[90, -180, 0, 180]`, or leave out the key altogether to get global data. The data is saved in `msl_monthly_south_1979_2020.nc`. Provide the full path if you want it saved under another directory, e.g. `../data/msl_monthly_south_1979_2020.nc`.

In [ ]:
import cdsapi

dataset = "reanalysis-era5-single-levels-monthly-means"
request = {
    "product_type": ["monthly_averaged_reanalysis"],
    "variable": ["mean_sea_level_pressure"],
    "year": [
        "1979",
        "1980",
        "1981",
        "1982",
        "1983",
        "1984",
        "1985",
        "1986",
        "1987",
        "1988",
        "1989",
        "1990",
        "1991",
        "1992",
        "1993",
        "1994",
        "1995",
        "1996",
        "1997",
        "1998",
        "1999",
        "2000",
        "2001",
        "2002",
        "2003",
        "2004",
        "2005",
        "2006",
        "2007",
        "2008",
        "2009",
        "2010",
        "2011",
        "2012",
        "2013",
        "2014",
        "2015",
        "2016",
        "2017",
        "2018",
        "2019",
        "2020",
    ],
    "month": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"],
    "time": ["00:00"],
    "data_format": "netcdf",
    "download_format": "unarchived",
    "area": [0, -180, -90, 180],
}

os.chdir(data_path)
client = cdsapi.Client()
client.retrieve(dataset, request).download("msl_monthly_south_1979_2020.nc")
os.chdir(current_directory)

### Download daily data (2014-2020)

Now download the daily `msl` data from ERA5. This data could be downloaded using Anemoi, but it would be in a different resolution and gridding system (o320) than the data downloaded directly using the CDS API or `download_toolbox`, which means you'll have to regrid it. We'll use the `download_era5` command from the `download_toolbox` CLI, which will download ERA5 daily data on a 0.25 x 0.25 degree grid, like the monthly averages downloaded above. You can also download this using the CDS API directly, but `download_toolbox` simply seems a lot faster. You can call the command from this notebook with the following:

In [ ]:
import subprocess

os.chdir(data_path)
subprocess.run(
    [
        "download_era5",
        "-f",
        "DAY",
        "-o",
        "MONTH",
        "south",
        "2014-01-01",
        "2020-12-31",
        "psl",
        "",
    ]
)
os.chdir(current_directory)

> NB: this will save the data under `${data_path}/data/era5/psl/south/*nc`, in monthly grouped files. But it will also duplicate them under `${data_path}/data/era5/day/south/psl/*nc`. Apparently, there's supposed to be a "delete cache" option in the download_toolbox, which I couldn't find.

## Data checks

### Monthly data

In [ ]:
import xarray as xr

ds = xr.open_dataset(data_path / "msl_monthly_south_1979_2020.nc", engine="h5netcdf")
print(ds)
lat_step = abs(ds.latitude[1].item() - ds.latitude[0].item())
lon_step = abs(ds.longitude[1].item() - ds.longitude[0].item())
print(f"Resolution: {lat_step} x {lon_step}")

In [ ]:
import pandas as pd

times = pd.to_datetime(ds.valid_time.values)
print(f"Start: {times.min()}")
print(f"End: {times.max()}")
print(f"Number of months: {len(times)}")
expected = pd.date_range("1979-01-01", "2020-12-01", freq="MS")
missing = expected.difference(times)
print(f"Missing {len(missing)} months")

### Daily data

In [ ]:
import xarray as xr

ds = xr.open_dataset(data_path / "data/cds/psl/south/201405.nc", engine="h5netcdf")
print(ds)
lat_step = abs(ds.latitude[1].item() - ds.latitude[0].item())
lon_step = abs(ds.longitude[1].item() - ds.longitude[0].item())
print(f"Resolution: {lat_step} x {lon_step}")

In [ ]:
import xarray as xr

ds = xr.open_dataset(data_path / "msl_monthly_south_1979_2020.nc", engine="h5netcdf")
# Take the mean across years for each calendar month. Shape: (12, lat, lon)
clim = ds["msl"].groupby("valid_time.month").mean(dim="valid_time")
len(clim)

In [ ]:
clim.to_dataset(name="msl").to_netcdf(data_path / "data/climatology_12_month.nc", engine="h5netcdf")

## Calculate anomalies

In [ ]:
clim = xr.open_dataset(data_path / "data/climatology_12_month.nc", engine="h5netcdf")

In [ ]:
clim

In [ ]:
import xarray as xr
import glob
import os

# open clim file if not already loaded from previous
clim = xr.open_dataset(data_path / "data/climatology_12_month.nc", engine="h5netcdf")

# Daily files folder
input_folder = data_path / "data/cds/psl/south/"
output_folder = data_path / "data/era5/anom_psl/south/"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Get list of daily .nc files
daily_files = sorted(glob.glob(os.path.join(input_folder, "*.nc")))

for f in daily_files:
    print(f"Processing {f}")
    ds_daily = xr.open_dataset(f, engine="h5netcdf")

    # Extract month from the first time value
    month = ds_daily["time"].dt.month[0].item()

    # Select the corresponding climatology slice
    clim_month = clim.sel(month=month)

    # Subtract climatology from daily values
    anomaly = ds_daily["psl"] - clim_month["msl"]

    # Preserve original filename, save under new folder
    filename = os.path.basename(f)
    out_file = os.path.join(output_folder, filename)

    # Save anomaly
    anomaly.to_dataset(name="psl_anom").to_netcdf(out_file, engine="h5netcdf")

    print(f"Saved {out_file}")

## Visualize

In [ ]:
ds_anom = xr.open_dataset(data_path / "data/era5/anom_psl/south/201401.nc", engine="h5netcdf")
ds_mls = xr.open_dataset(data_path / "data/cds/psl/south/201401.nc", engine="h5netcdf")

psl_anom = ds_anom["psl_anom"].isel(time=0) / 100  # Convert Pa → hPa
ds_mls = ds_mls["psl"].isel(time=0) / 100  # Convert Pa → hPa

In [ ]:
antarctic_extent = [-180, 180, -90, -60]
arctic_extent = [-180, 180, 60, 90]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmocean


def plot_msl(msl_data, title):
    is_arctic = msl_data.latitude.mean().item() > 0
    proj = ccrs.NorthPolarStereo() if is_arctic else ccrs.SouthPolarStereo()

    plt.figure(figsize=(8, 8))
    ax = plt.axes(projection=proj)

    # Set appropriate extent for polar views
    if is_arctic:
        ax.set_extent(arctic_extent, crs=ccrs.PlateCarree())
    else:
        ax.set_extent(antarctic_extent, crs=ccrs.PlateCarree())

    # Basemap features
    ax.coastlines(resolution="50m", linewidth=1)
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.add_feature(cfeature.LAND, facecolor="lightgray")
    # ax.gridlines(draw_labels=False, linewidth=0.5, color='gray', alpha=0.5)

    # Pressure levels
    levels = np.linspace(msl_data.min().item(), msl_data.max().item(), 30)
    cs = ax.contourf(
        msl_data.longitude,
        msl_data.latitude,
        msl_data,
        levels=levels,
        transform=ccrs.PlateCarree(),
        cmap=cmocean.cm.diff,
        extend="both",
    )

    # Colorbar
    plt.colorbar(cs, orientation="horizontal", pad=0.05, aspect=50, label="MSLP (hPa)")

    ax.set_title(title)

    # Apply tight layout
    plt.tight_layout()
    plt.show()

In [ ]:
plot_msl(ds_mls, title="MSLP (Antarctic)")
plot_msl(psl_anom, title="MSLP Anomalies (Antarctic)")